In [1]:
from train_nep.config import get_nep_config
from train_nep.dataset import StructureDataset, collate_fn
from train_nep.model import NEP
from train_nep.alchemy import Alchemy
from torch.utils.data import DataLoader

config = get_nep_config()

train_dataset = StructureDataset(filepath="../Repository/PbTe/train.xyz", para=config)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=False, collate_fn=collate_fn)

model = NEP(config)

alchemy = Alchemy(
    model=model,
    training_set=train_loader,
    save_path="../Repository/PbTe/nep_model.pt"
)

alchemy.fit(epochs=100)
alchemy.load("../Repository/PbTe/nep_model.pt")

wandb: Currently logged in as: 602585882 (train_nep) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


开始训练，总共 100 轮
设备: cpu
模型参数数量: 3034
--------------------------------------------------
  [Batch 1] Loss = 4.561421 | Energy = 4.188074 | Forces = 0.373347 | Virial = 0.000000
  [Batch 2] Loss = 4.483187 | Energy = 4.127036 | Forces = 0.356152 | Virial = 0.000000
  [Batch 3] Loss = 4.364891 | Energy = 4.068189 | Forces = 0.296702 | Virial = 0.000000
  [Batch 4] Loss = 4.291010 | Energy = 4.003034 | Forces = 0.287976 | Virial = 0.000000
  [Batch 5] Loss = 4.281257 | Energy = 3.897299 | Forces = 0.383958 | Virial = 0.000000
Epoch   1/100: Train Loss = 4.396353 | Energy = 4.056726 | Forces = 0.339627 | Virial = 0.000000
  [Batch 1] Loss = 4.196388 | Energy = 3.825942 | Forces = 0.370447 | Virial = 0.000000
  [Batch 2] Loss = 4.118040 | Energy = 3.763638 | Forces = 0.354402 | Virial = 0.000000
  [Batch 3] Loss = 4.004118 | Energy = 3.705616 | Forces = 0.298503 | Virial = 0.000000
  [Batch 4] Loss = 3.935577 | Energy = 3.640070 | Forces = 0.295506 | Virial = 0.000000
  [Batch 5] Loss = 3.9183

Loss/train_energy,█▇▅▅▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/train_forces,████▇▇▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/train_total,█▇▆▆▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/train_virial,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
Loss/train_energy,0.00877
Loss/train_forces,0.12559
Loss/train_total,0.13436
Loss/train_virial,0
epoch,100



训练完成!
最佳验证损失: inf
✓ 已加载检查点: ../Repository/PbTe/nep_model.pt


In [2]:
import torch
from wizard.io import read_xyz
from train_nep.interface import NEPCalculator
from train_nep.config import get_nep_config

test_atoms = read_xyz("../Repository/PbTe/train.xyz")[0]  

para = get_nep_config()  
nep_calc = NEPCalculator(model_path="../Repository/PbTe/nep_model.pt", para=para) 
test_atoms.calc = nep_calc

pred_energy = test_atoms.get_potential_energy()
pred_forces = test_atoms.get_forces()

print(f"N_atoms: {len(test_atoms)} atoms")
print(f"Chemical formula: {test_atoms.get_chemical_formula()}")
print(f"Predicted energy: {pred_energy:.6f} eV")
print(f"Energy per atom: {pred_energy/len(test_atoms):.6f} eV/atom")
print(f"Forces shape: {pred_forces.shape}")

if 'energy' in test_atoms.info:
    dft_energy = test_atoms.info['energy']
    print(f"\nComparison with true (DFT) values:")
    print(f"Energy: {dft_energy/len(test_atoms):.6f} eV")
    print(f"Energy error: {abs(pred_energy - dft_energy)/len(test_atoms):.6f} eV/atom")

if 'forces' in test_atoms.info:
    dft_forces = test_atoms.info['forces']
    diff = torch.tensor(pred_forces - dft_forces)  # [N,3]
    mae_force = diff.abs().mean()
    print(f"Force MAE: {mae_force:.6f} eV/Å")

N_atoms: 250 atoms
Chemical formula: Pb125Te125
Predicted energy: -934.769287 eV
Energy per atom: -3.739077 eV/atom
Forces shape: (250, 3)

Comparison with true (DFT) values:
Energy: -3.748764 eV
Energy error: 0.009687 eV/atom
Force MAE: 0.167928 eV/Å
